In [70]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from pytorch_dataset import VegetableDataset
from model import VegetableModel

In [71]:
def count_images_in_folder(folder_path):
    count = 0
    # Traverse through all subdirectories and files
    for root, dirs, files in os.walk(folder_path):
        count += len(files)
    return count

def count_images_in_subfolders(base_folder):
    counts = {}
    for subfolder in os.listdir(base_folder):
        subfolder_path = os.path.join(base_folder, subfolder)
        if os.path.isdir(subfolder_path):
            image_count = count_images_in_folder(subfolder_path)
            counts[subfolder] = image_count
    return counts

In [72]:
# Paths
train_folder = "./data/train"
valid_folder = "./data/validation"
test_folder = "./data/test"

In [73]:
# Count images
train_counts = count_images_in_subfolders(train_folder)
valid_counts = count_images_in_subfolders(valid_folder)
test_counts = count_images_in_subfolders(test_folder)

print(f"Train dataset counts: {train_counts}")
print(f"Validation dataset counts: {valid_counts}")
print(f"Test dataset counts: {test_counts}")

Train dataset counts: {'Bean': 1000, 'Bitter_Gourd': 1000, 'Bottle_Gourd': 1000, 'Brinjal': 1000, 'Broccoli': 1000, 'Cabbage': 1000, 'Capsicum': 1000, 'Carrot': 1000, 'Cauliflower': 1000, 'Cucumber': 1000, 'Papaya': 1000, 'Potato': 1000, 'Pumpkin': 1000, 'Radish': 1000, 'Tomato': 1000}
Validation dataset counts: {'Bean': 200, 'Bitter_Gourd': 200, 'Bottle_Gourd': 200, 'Brinjal': 200, 'Broccoli': 200, 'Cabbage': 200, 'Capsicum': 200, 'Carrot': 200, 'Cauliflower': 200, 'Cucumber': 200, 'Papaya': 200, 'Potato': 200, 'Pumpkin': 200, 'Radish': 200, 'Tomato': 200}
Test dataset counts: {'Bean': 200, 'Bitter_Gourd': 200, 'Bottle_Gourd': 200, 'Brinjal': 200, 'Broccoli': 200, 'Cabbage': 200, 'Capsicum': 200, 'Carrot': 200, 'Cauliflower': 200, 'Cucumber': 200, 'Papaya': 200, 'Potato': 200, 'Pumpkin': 200, 'Radish': 200, 'Tomato': 200}


In [74]:
# Total counts
total_train_images = sum(train_counts.values())
total_valid_images = sum(valid_counts.values())
total_test_images = sum(test_counts.values())

print(f"Total training images: {total_train_images}")
print(f"Total validation images: {total_valid_images}")
print(f"Total test images: {total_test_images}")

Total training images: 15000
Total validation images: 3000
Total test images: 3000


In [85]:
# Configs
num_classes = 15
batch_size = 8
num_epochs = 20
train_losses, val_losses = [], []

In [86]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [87]:
# Data augmentation and Normalization

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop((128, 128), scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [88]:
# Dataset
train_dataset = VegetableDataset(train_folder, transform=transform)
val_dataset = VegetableDataset(valid_folder, transform=transform)
test_dataset = VegetableDataset(test_folder, transform=transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [89]:
# Model
model = VegetableModel(num_classes=num_classes)
model.to(device) 


VegetableModel(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_s

In [90]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [91]:
# Training
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0 
    all_labels = []
    all_predictions = []

    for images, labels in tqdm(train_loader, desc="Training loop"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() 
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # accuracy calculation
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

        running_loss += loss.item() * labels.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = accuracy_score(all_labels, all_predictions)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation loop"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            running_loss += loss.item() * labels.size(0)

    val_loss = running_loss / len(val_loader.dataset)
    val_accuracy = accuracy_score(all_labels, all_predictions)
    val_losses.append(val_loss)

    # Results for each epoch
    print(f"Epoch {epoch}/{num_epochs} - Train loss: {train_loss:.4f}, Train accuracy: {train_accuracy:.4f}, Validation loss: {val_loss:.4f}, Validation accuracy: {val_accuracy:.4f}")

    # Save Model
    os.makedirs("model", exist_ok=True)
    torch.save(obj=model.state_dict(), f=f"model/vegetable_epoch_{epoch}.pth")  # Epoch sayısını dosya isminde gösteriyoruz


Validation loop: 100%|██████████| 375/375 [00:24<00:00, 15.41it/s]


Epoch 1/20 - Train loss: 0.4272, Train accuracy: 0.8711, Validation loss: 0.0972, Validation accuracy: 0.9703


Validation loop: 100%|██████████| 375/375 [00:24<00:00, 15.62it/s]


Epoch 2/20 - Train loss: 0.2035, Train accuracy: 0.9381, Validation loss: 0.1222, Validation accuracy: 0.9647


Validation loop: 100%|██████████| 375/375 [00:21<00:00, 17.75it/s]


...

Epoch 18/20 - Train loss: 0.0377, Train accuracy: 0.9886, Validation loss: 0.0308, Validation accuracy: 0.9920


Validation loop: 100%|██████████| 375/375 [00:24<00:00, 15.45it/s]


Epoch 19/20 - Train loss: 0.0511, Train accuracy: 0.9869, Validation loss: 0.0149, Validation accuracy: 0.9953


Validation loop: 100%|██████████| 375/375 [00:25<00:00, 14.96it/s]


Epoch 20/20 - Train loss: 0.0368, Train accuracy: 0.9891, Validation loss: 0.0900, Validation accuracy: 0.9790
